In [ ]:
!pip install qrcode

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 2.7 MB/s eta 0:00:00


In [ ]:
!apt install libzbar0

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  fonts-droid-fallback fonts-noto-mono fonts-urw-base35 ghostscript gsfonts imagemagick-6-common
  libdjvulibre-text libdjvulibre21 libfftw3-double3 libgs9 libgs9-common libidn12 libijs-0.35
  libjbig2dec0 libjxr-tools libjxr0 liblqr-1-0 libmagickcore-6.q16-6 libmagickcore-6.q16-6-extra
  libmagickwand-6.q16-6 libv4l-0 libv4lconvert0 libwmflite-0.2-7 poppler-data
Suggested packages:
  fonts-noto fonts-freefont-otf | fonts-freefont-ttf fonts-texgyre ghostscript-x libfftw3-bin
  libfftw3-dev inkscape poppler-utils fonts-japanese-mincho | fonts-ipafont-mincho
  fonts-japanese-gothic | fonts-ipafont-gothic fonts-arphic-ukai fonts-arphic-uming fonts-nanum
The following NEW packages will be installed:
  fonts-droid-fallback fonts-noto-mono fonts-urw-base35 ghostscript gsfonts imagemagick-6-common
  libdjvulibre-text libdjvulibre21 libfftw3-doub

In [ ]:
!pip install pyzbar

In [ ]:
!pip install pycryptodome

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 34.2 MB/s eta 0:00:00


In [ ]:
import random
import json
import qrcode
import base64
import io
import os
from pyzbar.pyzbar import decode
from PIL import Image
from Crypto.Cipher import AES
from Crypto.Hash import HMAC, SHA256

# --------------- Helper Functions ---------------
def manual_random_bytes(n):
    """Generates n random bytes manually."""
    return bytes([random.randint(0, 255) for _ in range(n)])

def split_data(data, chunk_size):
    """Splits large data into chunks of `chunk_size` bytes."""
    return [data[i:i+chunk_size] for i in range(0, len(data), chunk_size)]

# --------------- AES-GCM Encryption ---------------
def generate_keys():
    """Generate AES and HMAC keys."""
    aes_key = manual_random_bytes(32)  # AES-256 key
    hmac_key = manual_random_bytes(32)  # HMAC-SHA256 key
    return aes_key, hmac_key

def encrypt_audio(audio_path, aes_key, hmac_key):
    """Encrypts an audio file without quality loss and returns encrypted JSON data."""
    with open(audio_path, "rb") as f:
        audio_data = f.read()

    nonce = manual_random_bytes(12)
    cipher = AES.new(aes_key, AES.MODE_GCM, nonce=nonce)
    ciphertext, auth_tag = cipher.encrypt_and_digest(audio_data)

    hmac = HMAC.new(hmac_key, ciphertext + nonce, SHA256).digest()

    encrypted_output = {
        "ciphertext": base64.b64encode(ciphertext).decode('utf-8'),
        "auth_tag": base64.b64encode(auth_tag).decode('utf-8'),
        "nonce": base64.b64encode(nonce).decode('utf-8'),
        "hmac": base64.b64encode(hmac).decode('utf-8')
    }

    return json.dumps(encrypted_output)

# --------------- QR Code Generation ---------------
def generate_multi_qr_audio(encrypted_data, max_chunk_size=1000):
    """Splits encrypted audio data into chunks and generates multiple QR codes."""
    chunks = split_data(encrypted_data, max_chunk_size)
    total_parts = len(chunks)

    for i, chunk in enumerate(chunks):
        chunk_data = json.dumps({"part": i + 1, "total": total_parts, "data": chunk})
        qr = qrcode.QRCode(version=None, error_correction=qrcode.constants.ERROR_CORRECT_L, box_size=10, border=5)
        qr.add_data(chunk_data)
        qr.make(fit=True)
        img = qr.make_image(fill="black", back_color="white")
        filename = f"secure_audio_qr_part_{i+1}.png"
        img.save(filename)
        print(f"QR Code {i+1}/{total_parts} saved as {filename}")

# --------------- QR Code Scanning & Reconstruction ---------------
def scan_qr_codes(qr_files):
    """Scans multiple QR codes and reconstructs the full encrypted data."""
    scanned_parts = {}

    for qr_file in qr_files:
        img = Image.open(qr_file)
        decoded_data = decode(img)
        if decoded_data:
            part_info = json.loads(decoded_data[0].data.decode())
            scanned_parts[part_info["part"]] = part_info["data"]

    if len(scanned_parts) == 0:
        print("No valid QR codes found!")
        return None

    total_parts = max(scanned_parts.keys())

    # Check if all parts are present
    if len(scanned_parts) != total_parts:
        print(f"Missing QR parts! Expected {total_parts}, but got {len(scanned_parts)}.")
        return None

    # Reconstruct full encrypted data
    full_data = "".join(scanned_parts[i] for i in range(1, total_parts + 1))
    return json.loads(full_data)  # Convert back to dictionary

# --------------- AES-GCM Decryption ---------------
def decrypt_audio(encrypted_data, aes_key, hmac_key, output_path="decrypted_audio.mp3"):
    """Decrypts AES-GCM encrypted audio data and verifies HMAC-SHA256."""
    ciphertext = base64.b64decode(encrypted_data["ciphertext"])
    auth_tag = base64.b64decode(encrypted_data["auth_tag"])
    nonce = base64.b64decode(encrypted_data["nonce"])
    hmac_received = base64.b64decode(encrypted_data["hmac"])

    hmac_calculated = HMAC.new(hmac_key, ciphertext + nonce, SHA256).digest()

    if hmac_received != hmac_calculated:
        raise ValueError("HMAC verification failed! QR Code may be tampered.")

    cipher = AES.new(aes_key, AES.MODE_GCM, nonce=nonce)
    decrypted_data = cipher.decrypt_and_verify(ciphertext, auth_tag)

    with open(output_path, "wb") as audio_file:
        audio_file.write(decrypted_data)

    print(f"Decrypted audio saved as {output_path}")

# --------------- Example Usage ---------------
if __name__ == "__main__":
    aes_key, hmac_key = generate_keys()
    audio_path = input("Enter the audio path to encrypt: ")

    # Encrypt audio without quality reduction
    encrypted_data = encrypt_audio(audio_path, aes_key, hmac_key)

    # Generate QR codes from encrypted audio data
    generate_multi_qr_audio(encrypted_data)

    # Scan QR codes and reconstruct encrypted data
    qr_files = [f for f in os.listdir() if f.startswith("secure_audio_qr_part_")]
    encrypted_data_scanned = scan_qr_codes(qr_files)

    if encrypted_data_scanned:
        decrypt_audio(encrypted_data_scanned, aes_key, hmac_key)


Enter the audio path to encrypt: /content/audioo.unknown
QR Code 1/73 saved as secure_audio_qr_part_1.png
QR Code 2/73 saved as secure_audio_qr_part_2.png
QR Code 3/73 saved as secure_audio_qr_part_3.png
QR Code 4/73 saved as secure_audio_qr_part_4.png
QR Code 5/73 saved as secure_audio_qr_part_5.png
QR Code 6/73 saved as secure_audio_qr_part_6.png
QR Code 7/73 saved as secure_audio_qr_part_7.png
QR Code 8/73 saved as secure_audio_qr_part_8.png
QR Code 9/73 saved as secure_audio_qr_part_9.png
QR Code 10/73 saved as secure_audio_qr_part_10.png
QR Code 11/73 saved as secure_audio_qr_part_11.png
QR Code 12/73 saved as secure_audio_qr_part_12.png
QR Code 13/73 saved as secure_audio_qr_part_13.png
QR Code 14/73 saved as secure_audio_qr_part_14.png
QR Code 15/73 saved as secure_audio_qr_part_15.png
QR Code 16/73 saved as secure_audio_qr_part_16.png
QR Code 17/73 saved as secure_audio_qr_part_17.png
QR Code 18/73 saved as secure_audio_qr_part_18.png
QR Code 19/73 saved as secure_audio_qr_part